In [3]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import requests

ModuleNotFoundError: No module named 'requests'

In [ ]:
import re
from bs4 import BeautifulSoup

In [ ]:
page_num=input("Enter the page number")


name=[]
processor=[]
storage=[]
battery=[]
ram=[]
rom=[]
xtd=[]
price=[]
Battery=[]
Processor=[]
for i in range(1,int(page_num)+1):
    
    url=("https://www.flipkart.com/search?q=mobiles&sid=tyy%2C4io&as=on&as-show=on&otracker=AS_QueryStore_OrganicAutoSuggest_1_6_na_na_na&otracker1=AS_QueryStore_OrganicAutoSuggest_1_6_na_na_na&as-pos=1&as-type=RECENT&suggestionId=mobiles%7CMobiles&requestId=b6e7f7ad-fa27-47a6-90de-61f680311991&as-backfill=on&p%5B%5D=facets.price_range.from%3D10000&p%5B%5D=facets.price_range.to%3DMax&page="+str(i))
    
    req=requests.get(url)
    
    content=BeautifulSoup(req.content,'html.parser')
    #print(content)
    
    page=content.find_all('ul',{"class":"_1xgFaf"})
    #print(processor)
    
    
    
    Phone_brand=content.find_all('div',{"class":"_4rR01T"})
    Price=content.find_all('div',{"class":"_30jeq3 _1_WHN1"})
    
    for i in Phone_brand:
        name.append(i.text)
        
    for i in Price:
        i=i.text
        price.append(re.sub("₹",'',i))
 #sub() is the substitute function. It returns the string obtained by replacing or substituting the leftmost non-overlapping occurrences of pattern in string by the replacement repl. If the pattern is not found, then the string is returned unchanged.   
    
    
    for i in page:
        details = i.find_all("li")
        
        try:
            storage.append(details[0].text)
        except:
            storage.append('nan')
 
        try:
            battery.append(details[3].text)
        except:
            battery.append('nan')
            
        try:
            if ('Processor' in details[3].text):
                processor.append(details[3].text)
            else:
                processor.append(details[4].text)
                
        except:
            processor.append('nan')
        


#print(processor)
#print(len(processor))

for i in battery:
    if re.match("\w{1,12}[\s]mAh",i):#\w -- (lowercase w) matches a "word" character: a letter or digit or underbar [a-zA-Z0-9_]. Note that although "word" is the mnemonic for this, it only matches a single word char, not a whole word. \W (upper case W) matches any non-word character.
        Battery.append(re.findall("\w{1,12}[\s]mAh",i))#%s specifically is used to perform concatenation of strings together. ... The number of values you want to append to a string should be equivalent to the number specified in parentheses after the % operator at the end of the string value. The following Python code illustrates the way of performing string formatting
    else:
        Battery.append('nan')


for i in processor:
    i=re.sub('[^\w\s]','',i)
    if("Processor" in i):
        Processor.append(i)
    else:
        Processor.append('nan')

    


for i in storage:

    if re.match("[0-9]+[\s]GB[\s]RAM",i):
        ram.append(re.findall("[0-9]+[\s]GB[\s]RAM",i))
    else:
        ram.append('nan')

  
     
    try:
        rom.append(re.findall("[0-9]+[\s]GB[\s]ROM",i))
    except:
        rom.append('nan')
        
             
    try:
        xtd.append(re.findall(r"Expandable[\s]Upto[\s][0-9]+[\s][A-Z]+",i))
    except:
        xtd.append('nan')
        

        
xtdm=[]

for i in xtd:
    if len(i)==0:
        xtdm.append('nan')
    else:
        xtdm.append(i)


ram = [''.join(ele) for ele in ram]  

rom = [''.join(ele) for ele in rom] 

Battery = [''.join(ele) for ele in Battery] 

xtdm = [''.join(ele) for ele in xtdm]     
        
data={"Name":name,"ROM":rom,"Extended_memory":xtdm,"Battery":Battery,"Processor":Processor,"Price":price,"RAM":ram}
df=pd.DataFrame(data)
print(df)
df.to_csv('mobil_ram.csv', index=False)

In [ ]:


import matplotlib.pyplot as plt
import seaborn as sn
import math, numpy



In [ ]:
df=pd.read_csv('mobil_ram.csv')
df.head()

In [4]:
df.info()

NameError: name 'df' is not defined

In [ ]:
df.isnull().sum()

In [ ]:
print(df['Price'].unique())
print(df['RAM'].unique())
print(df['ROM'].unique())
print(df['Extended_memory'].unique())

In [ ]:
#Cleaning

In [ ]:
df['Price']=df['Price'].str.replace(',','').astype(int)

In [ ]:
df['Extended_memory']=df['Extended_memory'].fillna('Expandable Upto 0 GB')

In [ ]:
df['Battery']=df['Battery'].str.split(' ').str.get(0).astype(int)

In [ ]:
df['RAM']=df['RAM'].str.split(' ').str.get(0).astype(int)

In [ ]:
df=df.dropna(subset= ['Processor','RAM','ROM','Battery'])

In [ ]:
df['ROM']=df['ROM'].str.split(' ').str.get(0).astype(int)

In [ ]:
df['Extended_memory']=df['Extended_memory'].str.split(' ').str.get(2).astype(int)

In [5]:
name=[]
for i in df['Name']:
    i=re.sub(r"\([^()]*\)", "", i)
    i=i.strip()
    name.append(i)
df['Name']=name
len(df['Name'])

NameError: name 'df' is not defined

In [ ]:
df.info()

In [ ]:
df=df.reset_index(drop=True)

In [ ]:
df.to_csv('mobile_detals.csv')

In [ ]:
#model

In [ ]:
print(df['RAM'].unique())
print(df['ROM'].unique())
print(df['Extended_memory'].unique())
print(df['Battery'].unique())


In [ ]:
sn.countplot(x='RAM',data=df)
plt.xlabel('RAM')
plt.ylabel('Number of Phones')
plt.show()

In [ ]:
df.groupby('RAM')['Price'].mean()

In [ ]:
sn.boxplot(x='RAM',y='Price',data=df)

In [ ]:
(df.Processor.unique())

In [ ]:
df.groupby('Processor')['Price'].mean()

In [ ]:
pd.pivot_table(df,index='RAM',values='Price')

In [6]:
pd.pivot_table(df, index = ['Name'], values = 'Price').sort_values('Price', ascending = False)

NameError: name 'df' is not defined

In [ ]:
(pd.pivot_table(df, index = ['Processor','RAM'], values = 'Price').sort_values('RAM', ascending = False))

In [ ]:
df['Name'].value_counts()

In [ ]:
pd.pivot_table(df, index = ['Name','Processor'], values = 'Price').sort_values('Price', ascending = False)

In [ ]:
pd.pivot_table(df, index = ['Name','Processor'], values = 'RAM').sort_values('RAM', ascending = False)

In [ ]:
len(name[name==1])


In [ ]:
len(df['Name'].unique())

In [ ]:
df.shape

In [ ]:
x=df.drop('RAM',axis=1)

In [ ]:
y=df['RAM']

In [ ]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=.2,random_state=0)

In [7]:
from sklearn.metrics import r2_score

ModuleNotFoundError: No module named 'sklearn'

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer

In [ ]:
ohe=OneHotEncoder()
ohe.fit(x[['Name','Processor']])

In [ ]:
column_trans = make_column_transformer((OneHotEncoder(categories=ohe.categories_),['Name','Processor']), remainder='passthrough')

In [ ]:
ohe.categories_

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline

In [8]:
forest = RandomForestClassifier(n_estimators = 30, criterion = 'entropy', random_state = 0)                         

NameError: name 'RandomForestClassifier' is not defined

In [ ]:
pipe=make_pipeline(column_trans,forest)

In [ ]:
pipe.fit(xtrain,ytrain)

In [ ]:
y_pred=pipe.predict(xtest)

In [ ]:
r2_score(ytest,y_pred)

In [9]:
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

ModuleNotFoundError: No module named 'sklearn'

In [ ]:
len(ytest)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.figure(figsize=(4,5))
sn.heatmap(confusion_matrix(ytest,y_pred),annot=True)
plt.xlabel("Actual")
plt.ylabel("Predict")
plt.show()

In [ ]:
knn=KNeighborsClassifier()
pipek=make_pipeline(column_trans,knn)
pipek.fit(xtrain,ytrain)

In [ ]:
y_pred=pipek.predict(xtest)

In [10]:
r2_score(ytest,y_pred)

NameError: name 'r2_score' is not defined

In [ ]:
lr=LogisticRegression()
pipel=make_pipeline(column_trans,lr)
pipel.fit(xtrain,ytrain)

In [ ]:
y_pred=pipel.predict(xtest)

In [ ]:
r2_score(ytest,y_pred)